
# Assignment: Linear Regression, Logistic Regression, and K-Means (From Scratch)

**Instructions**
- You are NOT allowed to use `scikit-learn` for model implementation, scaling.
- You may use it for implementation of clustering
- You may use: `numpy`, `matplotlib`, and standard Python libraries only.
- Every step (scaling, loss, gradients, optimization) must be implemented manually.
- Clearly comment your code and explain your reasoning in Markdown cells.


## Question 1: Linear Regression from Scratch (with Standardization and Regularization)

You are given a dataset `(X, y)`.

### Tasks
1. Implement **StandardScaler manually**:
   - Compute mean and standard deviation for each feature.
   - Standardize the features.
2. Implement **Linear Regression using Gradient Descent**.
3. Add **L2 Regularization (Ridge Regression)**.
4. Plot:
   - Loss vs iterations
   - True vs predicted values

Do NOT use `sklearn`.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt


In [ ]:

# Implement StandardScaler manually ,  first read about it, how it works and then implement it 
class StandardScalerManual:
    def fit(self, X):
        self.mean = np.mean(X, axis=0)
        self.std = np.std(X, axis=0)
        self.std[self.std == 0] = 1
        return self
    
    def transform(self, X):
        return (X - self.mean) / self.std
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

In [ ]:

# Implement Linear Regression from scratch, here you have to also construct the regulization term coefficient of which will be
# denoted by l2_lambda 
# try to implement L1 regularization or atlease read about it and where it is used
class LinearRegressionManual:
    def __init__(self, lr=0.01, epochs=1000, l2_lambda=0.0):
        self.lr = lr
        self.epochs = epochs
        self.l2_lambda = l2_lambda

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.W = np.zeros(n_features)
        self.b = 0
        self.losses = []

        for _ in range(self.epochs):
            y_pred = X @ self.W + self.b
            error = y_pred - y

            dw = (1 / n_samples) * (X.T @ error) + (self.l2_lambda / n_samples) * self.W
            db = (1 / n_samples) * np.sum(error)

            self.W -= self.lr * dw
            self.b -= self.lr * db

            loss = (1 / (2 * n_samples)) * np.sum(error ** 2) + \
                   (self.l2_lambda / (2 * n_samples)) * np.sum(self.W ** 2)
            self.losses.append(loss)

    def predict(self, X):
        return X @ self.W + self.b



## Question 2: Logistic Regression from Scratch (with Standardization and Regularization)

You are given a binary classification dataset.

### Tasks
1. Reuse your **manual StandardScaler**.
2. Implement **Logistic Regression using Gradient Descent**.
3. Use:
   - Sigmoid function
   - Binary Cross Entropy loss
4. Add **L2 Regularization**.
5. Report:
   - Training loss curve
   - Final accuracy

Do NOT use `sklearn`.


In [1]:

#Implement sigmoid function as told in the lectures 
def sigmoid(z):
    z = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z))


In [ ]:

#Implement Logistic Regression from scratch and here also add the regularizaation term 
class LogisticRegressionManual:
    def __init__(self, lr=0.01, epochs=1000, l2_lambda=0.0):
        self.lr = lr
        self.epochs = epochs
        self.l2_lambda = l2_lambda

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.W = np.zeros(n_features)
        self.b = 0
        self.losses = []

        for _ in range(self.epochs):
            linear = X @ self.W + self.b
            y_hat = sigmoid(linear)

            eps = 1e-9
            loss = -(1 / n_samples) * np.sum(
                y * np.log(y_hat + eps) + (1 - y) * np.log(1 - y_hat + eps)
            ) + (self.l2_lambda / (2 * n_samples)) * np.sum(self.W ** 2)
            self.losses.append(loss)

            dw = (1 / n_samples) * (X.T @ (y_hat - y)) + (self.l2_lambda / n_samples) * self.W
            db = (1 / n_samples) * np.sum(y_hat - y)

            self.W -= self.lr * dw
            self.b -= self.lr * db

        return self

    def predict_proba(self, X):
        return sigmoid(X @ self.W + self.b)

    def predict(self, X):
        return (self.predict_proba(X) >= 0.5).astype(int)




## Question 3: K-Means Clustering from Scratch (Matrix Clustering)

You are given a **random matrix** `M` of shape `(n, m)`.

### Tasks
Implement K-Means clustering **from scratch** such that:

1. Input:
   - A random matrix `M`
   - Number of clusters `k`
2. Output:
   - `assignment_table`: a matrix of same shape as `M`, where each element stores the **cluster label**
   - `cookbook`: a dictionary (hashmap) where:
     - Key = cluster index
     - Value = list of **positions (i, j)** belonging to that cluster
   - `centroids`: array storing centroid values

You must cluster **individual elements**, not rows.


In [ ]:

# Implement K-Means for matrix elements
#CAN USE SK-LEARN FOR THIS TASK AS THIS TASK WILL HELP US DIRECTLY IN OUR PROJECT ! 
def kmeans_matrix(M, k, max_iters=100):
    '''
    Returns:
    assignment_table: same shape as M, contains cluster labels
    cookbook: dict -> cluster_id : list of (i, j) positions
    centroids: numpy array of centroid values
    '''
    n, m = M.shape
    values = M.flatten()
    indices = [(i, j) for i in range(n) for j in range(m)]

    centroids = np.random.choice(values, k, replace=False)

    assignment_table = np.zeros((n, m), dtype=int)

    for _ in range(max_iters):
        clusters = {i: [] for i in range(k)}

        for idx, val in enumerate(values):
            distances = np.abs(val - centroids)
            cluster_id = np.argmin(distances)
            clusters[cluster_id].append(idx)
            i, j = indices[idx]
            assignment_table[i, j] = cluster_id

        new_centroids = centroids.copy()
        for cid in range(k):
            if clusters[cid]:
                new_centroids[cid] = np.mean([values[i] for i in clusters[cid]])

        if np.allclose(centroids, new_centroids):
            break
        centroids = new_centroids

    cookbook = {}
    for cid in range(k):
        cookbook[cid] = [indices[i] for i in clusters[cid]]

    return assignment_table, cookbook, centroids




## Submission Guidelines
- Submit the completed `.ipynb` file.
- Clearly label all plots and outputs.
- Code readability and correctness matter.
- Partial credit will be given for logically correct implementations.

**Bonus**
- Compare convergence with and without standardization.
- Try different values of regularization strength.
